In [149]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import warnings
from datetime import datetime
from contextlib import closing


# 불필요한 경고 메시지를 숨깁니다.
warnings.filterwarnings('ignore')

print(f'completed to load library. [{datetime.now()}]')

completed to load library. [2025-06-11 18:00:09.391358]


In [166]:
def load_data_by_db(db_file_path, last_round, length):
    import sqlite3
    from contextlib import closing
    with closing(sqlite3.connect(db_file_path)) as conn:
        with closing(conn.cursor()) as cur:
            results = []
            first_bonus = 0
            query = f'select * from results where round>={last_round-length} and round <={last_round}'
            datas = cur.execute(query).fetchall()
            for data in datas:
                if first_bonus == 0:
                    first_bonus = data[2]
                results.append((data[1], [int(i) for i in data[2].split(',')], data[3]))
            return results, first_bonus


def create_randomforest_db(db_file_path, verbose=1):
    columns = ["id INTEGER PRIMARY KEY AUTOINCREMENT",
               "n_estimator INTEGER",
               "data_length INTEGER",
               "random_state INTEGER",
               "rounds TEXT",
               "matched_cnts TEXT",
               "sum_val INTEGER",
               "version TEXT"]
    # (n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val)
    query = f'CREATE TABLE IF NOT EXISTS rndforest ({",".join(columns)})'
    if verbose > 0:
        print('query=', query)
    import sqlite3
    from contextlib import closing
    with closing(sqlite3.connect(db_file_path)) as conn:
        conn.execute(query)


"""
(25, 400, 2400, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
"""
def insert_randomforest_db(db_file_path, version, db_datas, auto_commit=True, verbose=0):
    """ insert_randomforest_db """
    import sqlite3
    from contextlib import closing
    from datetime import datetime
    if version == 0:
        version = str(datetime.now().timestamp())
    with closing(sqlite3.connect(db_file_path)) as conn:
        col_dicts = {"n_estimator": db_datas[0],
                     "data_length": db_datas[1],
                     "random_state": db_datas[2],
                     "rounds": ",".join([str(i) for i in db_datas[3][0]]),
                     "matched_cnts": ",".join([str(i) for i in db_datas[3][1]]),
                     "sum_val": db_datas[4],
                     "version": version
                     }
        columns = col_dicts.keys()
        values=list(col_dicts.values())
        column_val=["?" for i in range(len(columns))]
        query = f"INSERT INTO rndforest ({','.join(columns)}) "\
                f"VALUES ({','.join(column_val)})"
        with closing(conn.cursor()) as cursor:
            if verbose > 0:
                print(f'query={query}')
                print('-'*30)
            cursor.execute(query, values)
        if auto_commit:
            conn.commit()


In [161]:
create_randomforest_db('../db/metrics.db')

query= CREATE TABLE IF NOT EXISTS rndforest (id INTEGER PRIMARY KEY AUTOINCREMENT,n_estimator INTEGER,data_length INTEGER,random_state INTEGER,rounds TEXT,matched_cnts TEXT,sum_val INTEGER,version TEXT)


In [80]:
def predict(train_X, n_estimators=100, random_state=350, verbose=0):
    # 데이터를 Pandas DataFrame으로 변환합니다.
    df = pd.DataFrame(train_X, columns=['회차', '번호1', '번호2', '번호3', '번호4', '번호5', '번호6'])
    df = df.sort_values(by='회차').reset_index(drop=True)

    if verbose > 0:
        print("--- 로또 당첨 번호 이력 ---")
        print(df)

    # 기계 학습을 위한 데이터 준비
    # 각 회차의 당첨 번호(X)와 바로 다음 회차의 당첨 번호(y)를 사용합니다.
    X = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']]
    y = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].shift(-1)

    # 마지막 행은 다음 회차 데이터가 없으므로 제거합니다.
    X = X[:-1]
    y = y.dropna()
    
    if verbose > 0:
        print("\n--- 훈련 데이터 (X) ---")
        print(X.head())
        print("\n--- 정답 데이터 (y) ---")
        print(y.head())

    # 머신러닝 모델 선택 및 훈련
    # 랜덤 포레스트 회귀 모델을 사용합니다.
    # the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
    # the n_estimators parameter specifies the number of decision trees in the forest. 
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state) # n_estimators: 만들 트리의 개수
    model.fit(X, y)

    # 예측할 회차의 이전 회차 데이터 (1173회차)
    last_draw = df.iloc[-1][['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.reshape(1, -1)

    if verbose > 0:
        print(f"\n--- {df.iloc[-1]['회차']}회차 데이터로 다음 회차 예측 ---")
        print(last_draw)

    # 다음 번호 예측
    predicted_numbers_set = []
    trial = 5
    for i in range(trial):
        predicted_numbers_float = model.predict(last_draw)
        # 예측된 번호 처리
        # 1. 소수점을 반올림하여 정수로 만듭니다.
        # 2. 1~45 사이의 값으로 보정합니다.
        # 3. 중복된 번호를 제거하고 6개를 선택합니다.
        predicted_numbers = set()
        for num in predicted_numbers_float[0]:
            # 반올림하여 정수로 변환
            int_num = int(round(num))
            # 1보다 작으면 1로, 45보다 크면 45로 보정
            if int_num < 1:
                int_num = 1
            elif int_num > 45:
                int_num = 45
            predicted_numbers.add(int_num)
        # 중복 제거 후 6개가 안되면, 부족한 만큼 다른 번호로 채웁니다.
        # (여기서는 가장 빈도가 높은 번호들 중 예측되지 않은 번호를 추가하는 방식을 사용)
        if len(predicted_numbers) < 6:
            all_numbers = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
            counts = pd.Series(all_numbers).value_counts()
    
            extra_needed = 6 - len(predicted_numbers)
            for num in counts.index:
                if extra_needed == 0:
                    break
                if num not in predicted_numbers:
                    predicted_numbers.add(num)
                    extra_needed -= 1
        # 최종 예측 번호를 정렬하여 출력
        final_prediction = sorted(list(predicted_numbers))[:6]
        predicted_numbers_set.append(final_prediction)
    return predicted_numbers_set


def test_prediction(round_nums,
                    actual_numbers,
                    bonus,
                    predicted_numbers_set,
                    n_estimators=500,
                    random_state=500,
                    only_one=True,
                    verbose=0):
    if verbose>0:
        print("======================================")
        print(f"\n--- 참고: {round_nums}회 실제 번호 ---")
        print(f"번호: {actual_numbers}, 보너스: {bonus}")

    matched_len = 0

    for final_prediction in predicted_numbers_set:
        # 예측 결과와 실제 결과 비교
        matching_numbers = set(final_prediction).intersection(set(actual_numbers))
        if verbose > 0:
            print(f"\n--- 예측과 실제 결과 비교 ---")
            print(f"일치하는 번호: {sorted(list(matching_numbers))}")
            print(f"일치 개수: {len(matching_numbers)}")
            print(F"예측 번호: {final_prediction}")
            print(f"n_estimators: {n_estimators}")
            print(f"random_state: {random_state}")
        matched_len = len(matching_numbers)
        if only_one:
            break
    return matched_len


def list_to_dict(item_lists):
    dicts = {}
    for item in item_lists:
        dicts[item[0]] = (item[1], item[2])
    return dicts


print(f'completed to define. #1 [{datetime.now()}]')

completed to define. #1 [2025-06-11 14:48:03.729748]


In [ ]:
verbose = 1

# 실제 번호
actual_numbers = list_to_dict(load_data_by_db(db_file_path='../db/metrics.db',
                                              last_round=1175,
                                              first_round=1172))

if verbose > 0:
    print('actual_numbers', actual_numbers)

data_list_cnt = 3
data_length = 25
results_set=[]

print(f'completed to set env. [{datetime.now()}]')

**Notice**
- the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
- the n_estimators parameter specifies the number of decision trees in the forest. This parameter is crucial as it directly impacts the model's performance and computational cost.

In [ ]:
def predit_and_test_parts(round, bonus, h_data, n_estimators, random_state, verbose=0):
    """ predit_and_test_parts """
    train_X = []
    test_X = []
    for i in range(len(h_data) - 1):
        train_X.append(h_data[i][1])
    test_X = h_data[len(h_data) - 1][1]
    predicted_numbers_set = predict(train_X=train_X,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=verbose)
    predict_len1=test_prediction(round,
                                 test_X,
                                 bonus,
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose)
    return predicted_numbers_set, predict_len1


def predict_and_test(round, bonus, h_data, n_estimators, random_state, verbose=0):
    """ predict_and_test """
    _, predict_len = predit_and_test_parts(round, bonus, h_data, n_estimators, random_state, 0)
    return predict_len


print(f'completed to define. #2 [{datetime.now()}]')

In [82]:
results_set = []

print(f'completed to reset results_set. #2 [{datetime.now()}]')

completed to reset results_set. #2 [2025-06-11 14:48:04.820160]


In [115]:
### Prepare and train, predict
def main_process(n_estimators,
                 last_rounds,
                 data_lengths,
                 random_state_gap=100,
                 random_state_begin=0,
                 random_state_end=300000,
                 db_file_path='../db/metrics.db'):
    predict_lens = {}
    for n_estimator in n_estimators:
        predict_lens[n_estimator] = {}
        for last_round in last_rounds:
            cnt = 0
            for data_length in data_lengths:
                hist_data, first_bonus = load_data_by_db(db_file_path=db_file_path, last_round=last_round, length=data_length)
                for train_x in hist_data:
                    train_x[1].insert(0, train_x[0])
                if data_length not in predict_lens[n_estimator]:
                    predict_lens[n_estimator][data_length] = {}
                for random_state in range(random_state_begin, random_state_end, random_state_gap):
                    predict_len = predict_and_test(last_round, first_bonus, hist_data, n_estimator, random_state, 0)
                    if n_estimator not in predict_lens:
                        predict_lens[n_estimator] = {}
                    if random_state not in predict_lens[n_estimator][data_length]:
                        predict_lens[n_estimator][data_length][random_state] = [[last_round], [predict_len]]
                    else:
                        predict_lens[n_estimator][data_length][random_state][0].append(last_round)
                        predict_lens[n_estimator][data_length][random_state][1].append(predict_len)
                    cnt += 1
                    if cnt % 1000 == 0:
                        print(f'proceed [now={datetime.now()}, cnt={cnt}, last_round={last_round}, random_state={random_state}, data_length={data_length}]')
    return predict_lens

In [151]:
def print_predicts(predict_lens, sum_min=-1, sum_max=-1, verbose=0, write_to_file=False):
    result_set = []
    for n_estimator in predict_lens:
        for data_length in predict_lens[n_estimator]:
            for random_state in predict_lens[n_estimator][data_length]:
                sum_val = sum(predict_lens[n_estimator][data_length][random_state][1])
                if sum_min == -1 and sum_max == -1:
                    if verbose > 0:
                        print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                    result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                else:
                    if sum_min > 0 and sum_max > 0 and sum_val >= sum_min and sum_val <= sum_max:
                        if verbose > 0:
                            print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                        result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                    else:
                        if sum_min > 0 and sum_val >= sum_min:
                            if verbose > 0:
                                print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                            result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                        elif sum_max > 0 and sum_val <= sum_max:
                            if verbose > 0:
                                print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                            result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                if write_to_file:
                    with closing(open("metric_temp.txt", "at")) as fd:
                        fd.write(f'{n_estimator},{data_length},{random_state},')
                        fd.write(f'{"/".join([str(i) for i in predict_lens[n_estimator][data_length][random_state][0]])},')
                        fd.write(f'{"/".join([str(i) for i in predict_lens[n_estimator][data_length][random_state][0]])},')
                        fd.write(f'{sum_val}\n')

    return result_set


In [169]:
print(f'start   [now={datetime.now()}]')
n_estimators=[25, 50, 10]
# data_lengths = [26, 25, 24]
data_lengths = [40]
random_state_gap = 1
random_state_begin = 0
random_state_end = 1000
# last_rounds = [1175, 1174, 1173]
last_rounds = [1176, 1175, 1174, 1173]
predict_lens = main_process(n_estimators=n_estimators,
                            last_rounds=last_rounds,
                            data_lengths=data_lengths,
                            random_state_gap=random_state_gap,
                            random_state_begin=random_state_begin,
                            random_state_end=random_state_end)

print(f'completed [now={datetime.now()}]')

start   [now=2025-06-11 18:30:51.798931]
proceed [now=2025-06-11 18:31:25.411891, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 18:31:59.764289, cnt=1000, last_round=1175, random_state=999, data_length=40]
proceed [now=2025-06-11 18:32:34.317706, cnt=1000, last_round=1174, random_state=999, data_length=40]
proceed [now=2025-06-11 18:33:08.904765, cnt=1000, last_round=1173, random_state=999, data_length=40]
proceed [now=2025-06-11 18:34:09.219086, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 18:35:09.095646, cnt=1000, last_round=1175, random_state=999, data_length=40]
proceed [now=2025-06-11 18:36:08.806608, cnt=1000, last_round=1174, random_state=999, data_length=40]
proceed [now=2025-06-11 18:37:08.596389, cnt=1000, last_round=1173, random_state=999, data_length=40]
proceed [now=2025-06-11 18:37:25.841677, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 18:37:43.249260, 

In [174]:
result_set = print_predicts(predict_lens=predict_lens, sum_min=1, write_to_file=False)

In [ ]:
print(result_set)

In [176]:
run_this=True
old_versions = ["T_01_01"]
version = "T_01_02"
if run_this:
    for result in result_set:
        insert_randomforest_db('../db/metrics.db', version=version, db_datas=result, verbose=0)

In [146]:
for result in result_set:
    if result[4] > 6:
        print(result)

(25, 400, 670, [[1176, 1175, 1174, 1173], [2, 2, 0, 3]], 7)
(25, 400, 1210, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 2190, [[1176, 1175, 1174, 1173], [1, 2, 2, 2]], 7)
(25, 400, 2400, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 4900, [[1176, 1175, 1174, 1173], [1, 2, 3, 1]], 7)
(25, 400, 5480, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 5850, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 6370, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 6680, [[1176, 1175, 1174, 1173], [2, 1, 3, 2]], 8)
(25, 400, 8350, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 11270, [[1176, 1175, 1174, 1173], [1, 2, 3, 1]], 7)
(25, 400, 11920, [[1176, 1175, 1174, 1173], [2, 1, 1, 3]], 7)
(25, 400, 12170, [[1176, 1175, 1174, 1173], [1, 2, 2, 2]], 7)
(25, 400, 12230, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 13220, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 14750, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 16160, [[

In [13]:
print(f'check answer : [{datetime.now()}]')
for result in results_set:
    if result[2][0] > 3 or result[2][1] > 3:
        print(result)

check answer : [2025-06-09 11:35:00.484062]


In [12]:
n_estimators=45
begin_pos=0
end_pos=600000
print(f'start   [now={datetime.now()}]')
for minus_value in [2, 1, 0]:
    cnt = 0
    for random_state in range(begin_pos, end_pos, 100):
        predict_lens = predict_and_test(minus_value, n_estimators, random_state, 0)
        if predict_lens[0] > 2 and predict_lens[1] > 2:
            results_set.append((n_estimators, random_state, predict_lens, minus_value))
            print('found = ', n_estimators, random_state, predict_lens, minus_value)
        cnt += 1
        if cnt % 1000 == 0:
            print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start   [now=2025-06-11 00:42:42.831085]
proceed [now=2025-06-11 00:44:28.745292, 1000]
proceed [now=2025-06-11 00:46:14.410815, 2000]
proceed [now=2025-06-11 00:48:00.175914, 3000]
proceed [now=2025-06-11 00:49:45.904667, 4000]
proceed [now=2025-06-11 00:51:31.441960, 5000]
proceed [now=2025-06-11 00:53:17.201224, 6000]
proceed [now=2025-06-11 00:55:03.388433, 1000]
proceed [now=2025-06-11 00:56:49.462688, 2000]
proceed [now=2025-06-11 00:58:35.198011, 3000]
proceed [now=2025-06-11 01:00:20.651806, 4000]
proceed [now=2025-06-11 01:02:04.516120, 5000]
proceed [now=2025-06-11 01:03:48.433397, 6000]
proceed [now=2025-06-11 01:05:32.586867, 1000]
proceed [now=2025-06-11 01:07:16.806856, 2000]
proceed [now=2025-06-11 01:09:01.147398, 3000]
proceed [now=2025-06-11 01:10:45.591498, 4000]
proceed [now=2025-06-11 01:12:30.120140, 5000]
proceed [now=2025-06-11 01:14:14.759978, 6000]
completed [now=2025-06-11 01:14:14.760198]


In [ ]:
n_estimators=6
begin_pos=100000
end_pos=500000
print(f'start   [now={datetime.now()}]')
for minus_value in [8, 7, 6, 5, 4, 3, 2, 1]:
    cnt = 0
    for random_state in range(begin_pos, end_pos, 100):
        predict_lens = predict_and_test(minus_value, n_estimators, random_state, 0)
        if predict_lens[0] > 1 and predict_lens[1] > 1 and predict_lens[2] > 1:
            results_set.append((n_estimators, random_state, predict_lens, minus_value))
            print('found = ', n_estimators, random_state, predict_lens, minus_value)
        cnt += 1
        if cnt % 1000 == 0:
            print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

```
(25, 219100, [3, 4], 0)
(25, 560100, [3, 4], 0)
(25, 322300, [3, 4], 5)
(25, 520500, [3, 4], 5)
(25, 148700, [3, 4], 6)
(25, 283500, [3, 4], 6)
(25, 358100, [3, 4], 6)
(25, 451600, [4, 3], 6)
(25, 489000, [4, 3], 6)
(25, 295500, [3, 4], 7)
(25, 241500, [3, 4], 8)
(45, 52100, [3, 4], 6)
(45, 587300, [3, 4], 4)
(10, 102500, [3, 4], 7)
(10, 113700, [4, 4], 6)
(10, 164200, [3, 4], 1)
(10, 225300, [3, 4], 8)
(10, 371400, [3, 4], 8)
(10, 255700, [3, 4], 7)
(10, 15200, [3, 4], 7)
(10, 63300, [3, 4], 6)
(6, 131400, [3, 4], 5)
(6, 195200, [4, 3], 5)
(6, 406100, [4, 3], 8)
(6, 325200, [4, 3], 7)
(6, 353200, [4, 3], 7)
(6, 473900, [4, 4], 6)
(6, 363000, [4, 3], 5)
[
(25, 451600, [4, 3], 6),
(25, 489000, [4, 3], 6),
(6, 473900, [4, 4], 6),
(10, 113700, [4, 4], 6),
(10, 15200, [3, 4], 7),
]

```

In [ ]:
for result in results_set:
    if result[2][0] > 3 or result[2][1] > 3:
        print(result)

In [ ]:
datas = [
    [45,208000,4,3,5],
    [45,101700,4,3,2],
    [45,101700,4,3,2],
    [45,185400,4,3,2],
    [45,96800,4,3,1],
    [45,58600,3,3,0],
    ]
datas = [
    (25, 451600, [4, 3], 6),
    (25, 489000, [4, 3], 6),
    (6, 473900, [4, 4], 6),
    (10, 113700, [4, 4], 6),
    (10, 15200, [3, 4], 7),
    ]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[3]
    results=[]
    print(f'start [now={datetime.now()}]')
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose=1)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')

In [ ]:
# datas = [
#     [45,208000,4,3,5],
#     [45,101700,4,3,2],
#     [45,185400,4,3,2],
#     [45,96800,4,3,1],
#     [45,58600,3,3,0],
#     ]
datas = [
    (25, 451600, [4, 3], 6),
    (25, 489000, [4, 3], 6),
    (6, 473900, [4, 4], 6),
    (10, 113700, [4, 4], 6),
    (10, 15200, [3, 4], 7),
    ]
print(f'start [now={datetime.now()}]')
results=[]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[3]    
    data_pos=0
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    results.append(predicted_numbers_set[0])
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[1],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')
print('Predicted Numbers.')
for result in results:
    print(result)

In [ ]:
print(results)

```
[
[45,208000,4,3,5],
[45,101700,4,3,2],
[45,101700,4,3,2],
[45,185400,4,3,2],
[45,96800,4,3,1],
    [45,58600,3,3,0],
]
```

In [ ]:
n_estimators=10
random_state=5000
data_pos=1
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0
                               )
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)
data_pos=2
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0)
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)

In [ ]:
[
    ([14, 17, 23, 27, 34, 36], [14, 17, 36], (45, 1000), (1160, 1173),
    ([13, 16, 23, 27, 34, 36], [36], (45, 1300), (1160, 1173)),
    ([11, 15, 21, 30, 36, 39], [11, 36, 39], (45, 1000), (1160, 1173),

]